# 机器翻译与数据集

In [1]:
import os
import torch
from d2l import torch as d2l

OSError: [WinError 1455] 页面文件太小，无法完成操作。 Error loading "E:\anaconda\envs\torch14\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

## 下载和预处理数据集

In [1]:
#法语到英语的翻译
#数据集里面本来就是一个个词
d2l.DATA_HUB['fra-eng'] = (d2l.DATA_URL + 'fra-eng.zip',
                           '94646ad1522d915e7b0f9296181140edcf86a4f5')

def read_data_nmt():
    """载入“英语－法语”数据集。"""
    data_dir = d2l.download_extract('fra-eng')
    with open(os.path.join(data_dir, 'fra.txt'), 'r') as f:
        return f.read()

raw_text = read_data_nmt()
print(raw_text[:75])

NameError: name 'd2l' is not defined

## 几个预处理步骤

In [2]:
#把标点符号留下来 把标点符号
def preprocess_nmt(text):
    """预处理“英语－法语”数据集。"""
    #如果标点前面没有空格的话，把标点前面加一个空格，这样就可以当作一个词切出来
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    out = [
        ' ' + char if i > 0 and no_space(char, text[i - 1]) else char
        for i, char in enumerate(text)]
    return ''.join(out)

text = preprocess_nmt(raw_text)
print(text[:80])

NameError: name 'raw_text' is not defined

## 词元化

In [1]:
#这里没有用字符，用的是词
def tokenize_nmt(text, num_examples=None):
    """词元化“英语－法语”数据数据集。"""
    source, target = [], []
    #text.split('\n')是把读到的内容根据’换行‘切分
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

source, target = tokenize_nmt(text)
source[:6], target[:6]

NameError: name 'text' is not defined

In [7]:
text_01 = '你是 我的老师'
print(text_01)
list_text_01 = text_01.split(' ')
print(list_text_01)

你是 我的老师
['你是', '我的老师']


## 绘制每个文本序列所包含的标记数量的直方图

In [8]:
d2l.set_figsize()
_, _, patches = d2l.plt.hist([[len(l)
                               for l in source], [len(l) for l in target]],
                             label=['source', 'target'])
for patch in patches[1].patches:
    patch.set_hatch('/')
d2l.plt.legend(loc='upper right');

NameError: name 'd2l' is not defined

## 词汇表

In [9]:
#'<pad>', '<bos>', '<eos>']分别是填充，开头，结尾
src_vocab = d2l.Vocab(source, min_freq=2,
                      reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(src_vocab)

NameError: name 'd2l' is not defined

## 序列样本都有一个固定的长度 截断或填充文本序列

In [10]:
#句子不能像语言模型那样切切
def truncate_pad(line, num_steps, padding_token):
    """截断或填充文本序列。"""
    if len(line) > num_steps:
        return line[:num_steps]
    return line + [padding_token] * (num_steps - len(line))

truncate_pad(src_vocab[source[0]], 10, src_vocab['<pad>'])

NameError: name 'src_vocab' is not defined

## 转换成小批量数据集用于训练

In [11]:
def build_array_nmt(lines, vocab, num_steps):
    """将机器翻译的文本序列转换成小批量。"""
    lines = [vocab[l] for l in lines]
    #加一个截止符，告诉模型什么时候这个句子结束了
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([
        truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
    #很多填充的东西都是没有意义的，valid_len是原始的有意义的
    return array, valid_len

## 训练模型

In [ ]:
#num_steps是每个句子的长度
def load_data_nmt(batch_size, num_steps, num_examples=600):
    """返回翻译数据集的迭代器和词汇表。"""
    text = preprocess_nmt(read_data_nmt())
    source, target = tokenize_nmt(text, num_examples)
    src_vocab = d2l.Vocab(source, min_freq=2,
                          reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = d2l.Vocab(target, min_freq=2,
                          reserved_tokens=['<pad>', '<bos>', '<eos>'])
    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)
    data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
    data_iter = d2l.load_array(data_arrays, batch_size)
    return data_iter, src_vocab, tgt_vocab

## 读出“英语－法语”数据集中的第一个小批量数据

In [13]:
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8)
for X, X_valid_len, Y, Y_valid_len in train_iter:
    print('X:', X.type(torch.int32))
    print('valid lengths for X:', X_valid_len)
    print('Y:', Y.type(torch.int32))
    print('valid lengths for Y:', Y_valid_len)
    break

NameError: name 'preprocess_nmt' is not defined

## 总结

- 生成的Y不再是一个标签，也是一个向量
- 填充 valid_len记录原始的长度